In [24]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for wandb from https://files.pythonhosted.org/packages/36/d5/215cac3edec5c5ac6e7231beb9d22466d5d4e4a132fa3a1d044f7d682c15/wandb-0.19.11-py3-none-win_amd64.whl.metadata
  Obtaining dependency information for docker-pycreds>=0.4.0 from https://files.pythonhosted.org/packages/f5/e8/f6bd1eee09314e7e6dee49cbe2c5e22314ccdb38db16c9fc72d2fa80d054/docker_pycreds-0.4.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for gitpython!=3.1.29,>=1.0.0 from https://files.pythonhosted.org/packages/1d/9a/4114a9057db2f1462d5c8f8390ab7383925fe1ac012eaa42402ad65c2963/GitPython-3.1.44-py3-none-any.whl.metadata
  Obtaining dependency information for sentry-sdk>=2.0.0 from https://files.pythonhosted.org/packages/f0/e5/da07b0bd832cefd52d16f2b9bbbe31624d57552602c06631686b93ccb1bd/sentry_sdk-2.29.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for setproctitle from h

In [23]:
pip install quantile-forest

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for quantile-forest from https://files.pythonhosted.org/packages/c2/e3/03f4cdb528045d8514abcfe99eee615fc90035139c8328831847084766c2/quantile_forest-1.4.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for scikit-learn>=1.5 from https://files.pythonhosted.org/packages/a1/a6/c5b78606743a1f28eae8f11973de6613a5ee87366796583fb74c67d54939/scikit_learn-1.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for threadpoolctl>=3.1.0 from https://files.pythonhosted.org/packages/32/d5/f9a850d79b0851d1d4ef6456097579a9005b31fea68726a4ae5f2d82ddd9/threadpoolctl-3.6.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/279.6 kB ? eta -:--:--
   ----- --------------------------------- 41.0/279.6 kB 991.0 kB/s eta 0:00:01
   ---------------------------------------  276.5/279.6 kB 3.4 MB/s eta 0:00:01
   -----------------------

In [5]:
import sys
from pathlib import Path

root = Path().resolve() / "topquartile"
if str(root) not in sys.path:
    sys.path.insert(0, str(root))

print("Added to PythonPath", root)

Added to PythonPath C:\Users\LO79RS\topquartile\topquartile


In [6]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor


In [1]:
from topquartile.modules.datamodule.partitions import PurgedTimeSeriesPartition
from topquartile.modules.datamodule.dataloader import DataLoader
from topquartile.modules.datamodule.transforms.covariate import TechnicalCovariateTransform
from topquartile.modules.datamodule.transforms.label import BinaryLabelTransform


In [4]:
# Defining Dataloader with covariates_may2025v2
covtrans_config = [(
TechnicalCovariateTransform,
dict(sma=[20, 30], ema=[20, 30], momentum_change=True, volatility=[20, 30]),
)]

labeltrans_config = [(BinaryLabelTransform, dict(label_duration=20, quantile=0.75))]

partition_config = dict(n_splits=5, gap=20, max_train_size=504, test_size=60)

dataloader = DataLoader(
data_id="covariates_may2025v2",
covariate_transform=covtrans_config,
label_transform=labeltrans_config,
partition_class=PurgedTimeSeriesPartition,
partition_kwargs=partition_config,
)

In [3]:
folds = dataloader.get_cv_folds()

Data not yet processed. Processing now...
Reading data from: C:\Users\LO79RS\topquartile\topquartile\data\covariates_may2025v2.csv
Found 134 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
THIS IS COLUMNS Index(['TOTAL_EQUITY', 'BOOK_VAL_PER_SH', 'REVENUE_PER_SH', 'RETURN_COM_EQY',
       'CUR_MKT_CAP', 'PX_LAST', 'TOT_DEBT_TO_TOT_ASSET',
       'TOT_DEBT_TO_TOT_EQY', 'BS_TOT_LIAB2', 'BS_TOT_ASSET', 'IS_EPS',
       'PX_HIGH', 'PX_LOW', 'PX_CLOSE_1D', 'PX_VOLUME', 'TURNOVER', 'ticker',
       'DVD_SH_12M'],
      dtype='object')
Applying label transformations globally to the dataset (before partitioning).
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


c:\Users\LO79RS\topquartile\topquartile\modules\datamodule\transforms\label.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.ihsg.index = pd.to_datetime(self.ihsg.index)


Data processing complete.
Partitioning data using PurgedTimeSeriesPartition for 5 splits across 85 tickers.
Fold 0: Train shape (42840, 29), Test shape (5100, 29)
Fold 1: Train shape (42840, 29), Test shape (5100, 29)
Fold 2: Train shape (42840, 29), Test shape (5100, 29)
Fold 3: Train shape (42840, 29), Test shape (5100, 29)
Fold 4: Train shape (42840, 29), Test shape (5100, 29)
Partitioning complete. Generated 5 CV folds.


In [6]:
folds = dataloader.get_cv_folds()
train_df, test_df = folds[0]
print(train_df.columns.tolist())

Data not yet processed. Processing now...
Reading data from: C:\Users\LO79RS\topquartile\topquartile\data\covariates_may2025v2.csv
Found 134 raw ticker names.
 Applying TechnicalCovariateTransform with params {'sma': [20, 30], 'ema': [20, 30], 'momentum_change': True, 'volatility': [20, 30]}
THIS IS COLUMNS Index(['TOTAL_EQUITY', 'BOOK_VAL_PER_SH', 'REVENUE_PER_SH', 'RETURN_COM_EQY',
       'CUR_MKT_CAP', 'PX_LAST', 'TOT_DEBT_TO_TOT_ASSET',
       'TOT_DEBT_TO_TOT_EQY', 'BS_TOT_LIAB2', 'BS_TOT_ASSET', 'IS_EPS',
       'PX_HIGH', 'PX_LOW', 'PX_CLOSE_1D', 'PX_VOLUME', 'TURNOVER', 'ticker',
       'DVD_SH_12M'],
      dtype='object')
Applying label transformations globally to the dataset (before partitioning).
 Applying BinaryLabelTransform with params {'label_duration': 20, 'quantile': 0.75} (globally)


c:\Users\LO79RS\topquartile\topquartile\modules\datamodule\transforms\label.py:51: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  self.ihsg.index = pd.to_datetime(self.ihsg.index)


Data processing complete.
Partitioning data using PurgedTimeSeriesPartition for 5 splits across 85 tickers.
Fold 0: Train shape (42840, 29), Test shape (5100, 29)
Fold 1: Train shape (42840, 29), Test shape (5100, 29)
Fold 2: Train shape (42840, 29), Test shape (5100, 29)
Fold 3: Train shape (42840, 29), Test shape (5100, 29)
Fold 4: Train shape (42840, 29), Test shape (5100, 29)
Partitioning complete. Generated 5 CV folds.
['TOTAL_EQUITY', 'BOOK_VAL_PER_SH', 'REVENUE_PER_SH', 'RETURN_COM_EQY', 'CUR_MKT_CAP', 'PX_LAST', 'TOT_DEBT_TO_TOT_ASSET', 'TOT_DEBT_TO_TOT_EQY', 'BS_TOT_LIAB2', 'BS_TOT_ASSET', 'IS_EPS', 'PX_HIGH', 'PX_LOW', 'PX_CLOSE_1D', 'PX_VOLUME', 'TURNOVER', 'ticker', 'sma_20', 'sma_30', 'ema_20', 'ema_30', 'volatility_20', 'volatility_30', 'roc_126', 'momentum_change', 'eq_returns_20', 'index_returns_20', 'excess_returns_20', 'label']


In [ ]:
train_df, test_df = folds[0]

train_df = train_df.dropna()
test_df = test_df.dropna()

y_train = train_df["label"]
y_test = test_df["label"]

X_train = train_df.drop(columns=["label"])
X_test = test_df.drop(columns=["label"])

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

Train shape: (5919, 28)
Test shape: (240, 28)


In [9]:
X_train.dtypes.value_counts()

float64     26
object       1
category     1
Name: count, dtype: int64

In [10]:
X_train.select_dtypes(include=["object"]).columns

Index(['TOTAL_EQUITY'], dtype='object')

In [11]:
X_train = X_train.drop(columns=["TOTAL_EQUITY"])
X_test = X_test.drop(columns=["TOTAL_EQUITY"])

In [14]:
X_train.dtypes.value_counts()

float64     26
category     1
Name: count, dtype: int64

In [15]:
X_train.select_dtypes(include=["category"]).columns

Index(['ticker'], dtype='object')

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
acc = accuracy_score(y_test, preds)

print(f"✅ Model accuracy on test set: {acc:.4f}")

ValueError: could not convert string to float: 'ADHI'